In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.1 MB/s eta 0:00:00


In [3]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from google.colab import drive
import pandas as pd
from datetime import datetime
from pandas.io.formats.info import pprint_thing
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
season_duration = pd.read_excel('/content/drive/MyDrive/DS105L/author_credit_nba/season_duration_1.xlsx')
season_duration_range = season_duration['duration'].to_list()
duration = []
for durations in season_duration_range:
  durations = durations.replace('\xa0',' ')
  durations = durations.replace('-', '–')
  start_date_str, end_date_str = durations.split(" – ")
  end_date_str = end_date_str[:-1] + re.sub('[^0-9]','', end_date_str[-1])
  
  # Parse the start and end dates into datetime objects
  start_date = datetime.strptime(start_date_str, "%B %d, %Y")
  end_date = datetime.strptime(end_date_str, "%B %d, %Y")

  # Format the start and end dates as strings with numerical month and year
  start_date_num = start_date.strftime("%Y%m%d")
  end_date_num = end_date.strftime("%Y%m%d")

  duration.append([int(start_date_num), int(end_date_num)])

season_duration['duration'] = duration

display(season_duration)

,season,duration
0,1950-51,"[19501031, 19510318]"
1,1951-52,"[19511101, 19520316]"
2,1952-53,"[19521031, 19530317]"
3,1953-54,"[19531030, 19540314]"
4,1954-55,"[19541030, 19550314]"
...,...,...
68,2018-19,"[20181016, 20190410]"
69,2019-20,"[20191022, 20200311]"
70,2020-21,"[20201222, 20210516]"
71,2021-22,"[20211019, 20220410]"


In [8]:
bbc_2010 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2010.csv')
bbc_2011 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2011.csv')
bbc_2012 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2012.csv')
bbc_2013 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2013.csv')
bbc_2014 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2014.csv')
bbc_2015 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2015.csv')
bbc_2016 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2016.csv')
bbc_2017 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2017.csv')
bbc_2018 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2018.csv')
bbc_2019 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2019.csv')
bbc_2020 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2020.csv')
bbc_2021 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2021.csv')
bbc_2022 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2022.csv')
bbc_2023 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_all_player_2010-2023/bbc_2023.csv')

def date_season_transfer(dataframe):

    time_list = dataframe['date'].to_list()
    date_list = []

    for idx, row in dataframe.iterrows():
        date = row['date']
        try:
            date_obj = datetime.strptime(date, '%d %B %Y')
        except ValueError:
            try:
                date_obj = datetime.strptime(date + ' 2023', '%d %B %Y')
            except ValueError:
                date_list.append(None)
                continue

        numerical_date = date_obj.strftime('%Y%m%d')
        date_list.append(int(numerical_date))

    season_duration_list = list(season_duration.iloc[:,1])

    season_transferred = []
    for new_date in date_list:
            found_season = False
            if new_date is not None:
                for period in duration:
                    if period[0] <= new_date <= period[1]:
                        row_index = [x[0] for x in season_duration_list].index(period[0])
                        season = season_duration.iloc[row_index, 0]
                        season_transferred.append(season)
                        found_season = True
                        break
            if not found_season:
                season_transferred.append('off-season news')

    dataframe['date'] = season_transferred
    dataframe = dataframe[dataframe['date'].notna()]
    dataframe = dataframe[dataframe['date'] != 'off-season news']

   
    return dataframe

drive.mount('/content/drive')

df10 = date_season_transfer(bbc_2010)
df11 = date_season_transfer(bbc_2011)
df12 = date_season_transfer(bbc_2012)
df13 = date_season_transfer(bbc_2013)
df14 = date_season_transfer(bbc_2014)
df15 = date_season_transfer(bbc_2015)
df16 = date_season_transfer(bbc_2016)
df17 = date_season_transfer(bbc_2017)
df18 = date_season_transfer(bbc_2018)
df19 = date_season_transfer(bbc_2019)
df20 = date_season_transfer(bbc_2020)
df21 = date_season_transfer(bbc_2021)
df22 = date_season_transfer(bbc_2022)
df23 = date_season_transfer(bbc_2023)




display(df11)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,title,date,url,player
0,Kevin Durant: Phoenix Suns forward out for thr...,2022-23,https://www.bbc.co.uk/sport/basketball/64912266,Kevin Durant
1,NBA: Kevin Durant scores 23 points on Phoenix ...,2022-23,https://www.bbc.co.uk/sport/basketball/64820742,Kevin Durant
2,NBA: Kevin Durant traded to Phoenix Suns as Ru...,2022-23,https://www.bbc.co.uk/sport/basketball/64593463,Kevin Durant
3,NBA All-Star Game: Giannis Antetokounmpo MVP a...,2020-21,https://www.bbc.co.uk/sport/basketball/56318297,Kevin Durant
4,NBA: Brooklyn Nets find 'solace' in wins after...,2022-23,https://www.bbc.co.uk/sport/basketball/63532635,Kevin Durant
...,...,...,...,...
15841,Sportsday Live - Monday 16 January,2011-12,http://news.bbc.co.uk/sport1/hi/front_page/967...,Deron Williams
15842,Chicago Cubs win World Series: 'Baseball will ...,2016-17,https://www.bbc.co.uk/sport/baseball/37857919,Deron Williams
15845,NFL off-season news and gossip,2009-10,http://news.bbc.co.uk/sport1/hi/other_sports/a...,Deron Williams
15846,Super Bowl XLII as it happened,2007-08,http://news.bbc.co.uk/sport1/hi/other_sports/a...,Deron Williams


In [13]:
df10 = df10[df10['date'] == '2010-11']
df11 = df10[df10['date'] == '2011-12']
df12 = df10[df10['date'] == '2012-13']
df13 = df10[df10['date'] == '2013-14']
df14 = df10[df10['date'] == '2014-15']
df15 = df10[df10['date'] == '2015-16']
df16 = df10[df10['date'] == '2016-17']
df17 = df10[df10['date'] == '2017-18']
df18 = df10[df10['date'] == '2018-19']
df19 = df10[df10['date'] == '2019-20']
df20 = df10[df10['date'] == '2020-21']
df21 = df10[df10['date'] == '2021-22']
df22= df10[df10['date'] == '2022-23']


In [14]:
df10.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2010_season.csv')
df11.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2011_season.csv')
df12.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2012_season.csv')
df13.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2013_season.csv')
df14.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2014_season.csv')
df15.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2015_season.csv')
df16.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2016_season.csv')
df17.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2017_season.csv')
df18.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2018_season.csv')
df19.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2019_season.csv')
df20.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2020_season.csv')
df21.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2021_season.csv')
df22.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2022_season.csv')
#df23.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2023_season.csv')

In [12]:
df10

,title,date,url,player
225,GB to face US in Olympic warm-up,2010-11,http://news.bbc.co.uk/sport1/hi/other_sports/b...,Kevin Durant
503,LeBron James becomes minority stakeholder in L...,2010-11,https://www.bbc.co.uk/sport/football/12994144,LeBron James
1384,Prototyping Weeknotes #39 (05/11/10),2010-11,http://www.bbc.co.uk/blogs/researchanddevelopm...,Chris Bosh
1533,Team-mates: Lee Mears & David Flatman,2010-11,http://news.bbc.co.uk/sport1/hi/front_page/927...,David Lee
1614,St David's Day: Leeks v daffs - true icons?,2010-11,https://www.bbc.co.uk/news/uk-wales-12602857,David Lee
...,...,...,...,...
16280,Future of GB basketball hangs in balance,2010-11,http://news.bbc.co.uk/sport1/hi/other_sports/b...,Drew Gooden
16567,NATO to control Libya no-fly zone,2010-11,http://news.bbc.co.uk/1/hi/programmes/gmt/9436...,Stephen Jackson
16780,Yeovil 0-1 Charlton,2010-11,http://news.bbc.co.uk/sport1/hi/football/eng_d...,Stephen Jackson
16829,Sloppy Nottingham defeated again,2010-11,http://news.bbc.co.uk/sport1/hi/rugby_union/91...,Stephen Jackson
